In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 문장 -> 임베딩 벡터(1차원 숫자 배열)
- openAI API의 키 OPENAI_API_KEY (text-embedding-3-large)를 .env에 추가
- upstage(https://console.upstage.ai) 의 키 UPSTAGE_API_KEY를 .env에 추가
# 1. 환경변수 load

In [3]:
from dotenv import load_dotenv
import os
load_dotenv(
   # dotenv_path='.env'
)
openai_key = os.getenv('OPENAI_API_KEY')
upstage_key = os.getenv('UPSTAGE_API_KEY')
print(openai_key[:2])
print(upstage_key[:2])

sk
up


# 2. 유사도 계산하는 방법
- 1. 유클리드 거리 : 두 벡터간 거리가 가까운지
- 2. 코사인 유사도 : 두 벡터간 방향이 유사한지
- 3. dot product : 두 벡터간 곱을 사용하여 거리와 방향을 모두 고려

In [4]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

# 3. openAI API의 embedding model 사용
- text-embedding-3-large

In [5]:
from openai import OpenAI
openai_client = OpenAI(
    # api_key=openai_key
)

In [22]:
# text-embedding-3-large(과금)
response = openai_client.embeddings.create(
    input= "The king is the prince's father",
    model="text-embedding-3-large"
)

In [23]:
response.usage

Usage(prompt_tokens=7, total_tokens=7)

In [24]:
king_vector = np.array(response.data[0].embedding) # The king is ~ 문장을 숫자로 바꾼 vec
print(king_vector.shape)
print(king_vector)

(3072,)
[ 0.03370763  0.03378944 -0.00436004 ...  0.01654019  0.01049273
 -0.00589065]


In [17]:
queen_response = openai_client.embeddings.create(
    input="The queen is the prince's mother",
    model="text-embedding-3-large"
)

In [18]:
queen_response.usage

Usage(prompt_tokens=7, total_tokens=7)

In [20]:
queen_vector = np.array(queen_response.data[0].embedding)
print(queen_vector.shape)
print(queen_vector)

(3072,)
[ 0.01908419  0.00117847 -0.00398764 ...  0.00576628  0.00822409
  0.01082648]


In [25]:
# king_vector와 queen_vector의 유사도
king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print("king문장과 queen문장의 유사도 :", king_queen_similarity)

king문장과 queen문장의 유사도 : 0.6953489600831544


In [29]:
slave_response = openai_client.embeddings.create(
    input = "The slave begs",
    model="text-embedding-3-large"
)

In [31]:
slave_vector = np.array(slave_response.data[0].embedding)
print(slave_vector.shape)
print(slave_vector)

(3072,)
[-0.00868506  0.00686     0.00754387 ... -0.00823628 -0.0112752
  0.00194153]


In [32]:
king_slave_similarity = cosine_similarity(king_vector, slave_vector)
print("king문장과 slave문장 유사도 :", king_slave_similarity)

king문장과 slave문장 유사도 : 0.18331621342902388


In [ ]:
# 한국어 문장을 벡터로 바꿔도 유사도가 비슷

In [33]:
response = openai_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="text-embedding-3-large"
)

In [37]:
kor_king_vector = np.array(response.data[0].embedding)
print(kor_king_vector.shape)
print(kor_king_vector)

(3072,)
[0.01460524 0.0081415  0.00299669 ... 0.00721534 0.01149217 0.00849063]


In [38]:
response = openai_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="text-embedding-3-large"
)

In [39]:
kor_queen_vector = np.array(response.data[0].embedding)
print(kor_queen_vector)

[-0.00633463  0.01295298  0.00534164 ... -0.00638844 -0.00050108
  0.0337521 ]


In [40]:
print("왕문장과 여왕문장의 유사도 :", cosine_similarity(kor_king_vector, kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.5744286874279492


In [41]:
response = openai_client.embeddings.create(
    input="노예가 구걸한다",
    model="text-embedding-3-large"
)

In [42]:
kor_slave_vector = np.array(response.data[0].embedding)
print(kor_slave_vector)

[ 0.00524604  0.0030029   0.00265426 ... -0.01131434 -0.00262631
  0.01126829]


In [43]:
print("왕문장과 노예문장의 유사도 :", cosine_similarity(kor_king_vector, kor_slave_vector))

왕문장과 노예문장의 유사도 : 0.13567770295253703


In [46]:
# king_vector와 kor_king_vector의 유사도
print("king문장과 왕문장의 유사도 :", cosine_similarity(king_vector, kor_king_vector))

king문장과 왕문장의 유사도 : 0.6248509347773605


# 4. upstage의 embedding model 사용
- 한국어에서는 openai 보다 성능이 좋다

In [48]:
upstage_client = OpenAI(
    api_key=upstage_key,
    base_url="https://api.upstage.ai/v1"
)

In [50]:
response = upstage_client.embeddings.create(
    input="The king is prince's father",
    model="embedding-query"
)

In [54]:
up_king_vector = np.array(response.data[0].embedding)
print(up_king_vector.shape)
print(up_king_vector)

(4096,)
[ 6.99263182e-05 -3.04292757e-02 -2.85108201e-03 ... -7.22163310e-03
  1.04733631e-02  3.70790288e-02]


In [55]:
response = upstage_client.embeddings.create(
    input="The queen is prince's mother",
    model="embedding-query"
)
up_queen_vector = np.array(response.data[0].embedding)

In [56]:
print("king문장과 queen문장의 유사도 :", cosine_similarity(up_king_vector, up_queen_vector))

king문장과 queen문장의 유사도 : 0.6694549910572132


In [57]:
response = upstage_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="embedding-query"
)

In [58]:
up_kor_king_vector = np.array(response.data[0].embedding)
print(up_kor_king_vector.shape)

(4096,)


In [ ]:
response = upstage_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="embedding-query"
)
up_kor_queen_vector = np.array()